In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# from signals import *
from frequencyestimator import *
from scipy.optimize import basinhopping, minimize
from tqdm.auto import tqdm

sns.set_style("whitegrid")
sns.despine(left=True, bottom=True)
sns.set_context("poster", font_scale = .45, rc={"grid.linewidth": 0.8})

<Figure size 640x480 with 0 Axes>

In [2]:
import numpy as np
from abc import ABCMeta, abstractmethod
import pickle
from scipy.linalg import toeplitz
from numba import njit

P0 = lambda n, theta: np.cos((2*n+1)*theta)**2
P1 = lambda n, theta: np.sin((2*n+1)*theta)**2
P0x = lambda n, theta: (1.0 + np.sin(2*(2*n+1)*theta))/2.0
P1x = lambda n, theta: (1.0 - np.sin(2*(2*n+1)*theta))/2.0

@njit
def get_ula_signal(q, idx, signal):
    p = np.outer(signal, np.conj(signal)).T.ravel()  # Compute outer product
    p = p[idx[0]]  # Restrict to indices
    cp = np.conj(p)
    for i in range(1, q):
        p = np.outer(p, cp).T.ravel() # Compute outer product iteratively
        p = p[idx[i]]  # Restrict to indices
    return p

class ULASignal(metaclass = ABCMeta):
    
    @abstractmethod
    def __init__(self):
        pass
    
    @abstractmethod
    def get_cov_matrix(self):
        pass
    
class TwoqULASignal(ULASignal):

    def __init__(self, M=None, ula=None, seed=None, C=1.2):
        '''
        Constructor for wrapper class around signal.
            ULA_signal_dict: either a dictionary containing the signal or path to a pickle file to load with the signal
        '''
        if seed: np.random.seed(seed)
        
        if isinstance(M, (list, np.ndarray)):
            self.M = M
            depths, n_samples = self._get_depths(self.M, C=C)
            self.depths = depths
            self.n_samples = n_samples
            self.q = len(self.M)//2 if len(self.M) % 2 == 0 else len(self.M)//2 + 1
            self.idx = self.get_idx()
        elif isinstance(ula, str):
            with open(ula, 'rb') as handle:
                self.idx, self.depths, self.n_samples, self.M = pickle.load(handle)
            self.q = len(self.M)//2 if len(self.M) % 2 == 0 else len(self.M)//2 + 1
        else:
            raise TypeError("Input ULA must by array of indices or path to pickle file")

    def save_ula(self, filename='ula.pkl'):
        with open(filename, 'wb') as handle:
            pickle.dump((self.idx, self.depths, self.n_samples, self.M), handle, protocol=pickle.HIGHEST_PROTOCOL)
        
    
    def get_cov_matrix(self, signal):
        '''
        This generates Eq. 13 in the paper DOI: 10.1109/DSP-SPE.2011.5739227 using the 
        technique from DOI:10.1109/LSP.2015.2409153
        '''
        self.ULA_signal = self.get_ula_signal(self.q, self.idx, signal)
        total_size = len(self.ULA_signal)
        ULA_signal = self.ULA_signal

        '''
        This uses the techinque from DOI:10.1109/LSP.2015.2409153
        '''
        subarray_col = ULA_signal[total_size//2:]
        subarray_row = np.conj(subarray_col)
        covariance_matrix = toeplitz(subarray_col, subarray_row)
        
        
        self.cov_matrix = covariance_matrix
        self.m = np.shape(self.cov_matrix)[0]
        self.R = covariance_matrix
        return covariance_matrix
    

    def get_cov_matrix_toeplitz(self, signal):
        '''
        This generates R tilde of DOI: 10.1109/LSP.2015.2409153 and only stores a column and row, which entirely 
        defines a Toeplitz matrix
        '''
        self.ULA_signal = get_ula_signal(self.q, self.idx, signal)
        total_size = len(self.ULA_signal)
        ULA_signal = self.ULA_signal
        
        subarray_col = ULA_signal[total_size//2:]
        subarray_row = np.conj(subarray_col)
        
        return subarray_col
    
    def get_idx(self):
        virtual_locations = []
        depths = self.depths
        q = self.q
        list_of_idx = []
        difference_matrix = np.zeros((len(depths), len(depths)), dtype=int)
        for r, rval in enumerate(depths):
            for c, cval in enumerate(depths):
                difference_matrix[r][c] = rval-cval
        depths0 = difference_matrix.flatten(order='F')
        depths0, idx = np.unique(depths0, return_index = True)
        new_depths = depths0
        list_of_idx.append(idx)

        virtual_locations.append(depths0)
        for i in range(q-1):
            difference_matrix = np.zeros((len(new_depths), len(depths0)), dtype=int)
            for r, rval in enumerate(new_depths):
                for c, cval in enumerate(depths0):
                    difference_matrix[r][c] = rval-cval
            new_depths = difference_matrix.flatten(order='F')
            new_depths, idx = np.unique(new_depths, return_index = True)
            virtual_locations.append(new_depths)

            if i<q-2:
                list_of_idx.append(idx)

        self.virtual_locations = virtual_locations

        difference_set = new_depths
        a = difference_set
        b = np.diff(a)
        b = b[:len(b)//2]
        try:
            start_idx = np.max(np.argwhere(b>1)) + 1
            list_of_idx.append(idx[start_idx:-start_idx])
        except:
            list_of_idx.append(idx)

        return list_of_idx

    def _get_depths(self, narray, C=1.2):
        physLoc = []
        n_samples = []

        r = (len(narray)-2)//2

        for i,m in enumerate(narray):
            c = int(np.prod(narray[:i]))
            for j in range(m):
                physLoc.append(j*c)

        physLoc = np.sort(list(set(physLoc)))

        for i in range(len(physLoc)):
            x = int((np.ceil(C*(len(physLoc)-i)))) # sims_99
            n_samples.append(x if x!=0 else 1)
        n_samples[0] = n_samples[0] * 2
        return physLoc, n_samples
    
    def estimate_signal(self, n_samples, theta, eta=0.0):
        depths = self.depths
        signals = np.zeros(len(depths), dtype = np.complex128)
        self.measurements = np.zeros(len(depths), dtype=np.double)
        for i,n in enumerate(depths):
            # Get the exact measuremnt probabilities
            p0 = P0(n, theta)
            p1 = P1(n, theta)

            p0x = P0x(n,theta)
            p1x = P1x(n,theta)

            # Get the "noisy" probabilities by sampling and adding a bias term that pushes towards 50/50 mixture
            eta_n = (1.0-eta)**(n+1) # The error at depth n increases as more queries are implemented
            p0_estimate = np.random.binomial(n_samples[i], eta_n*p0 + (1.0-eta_n)*0.5)/n_samples[i]
            p1_estimate = 1.0 - p0_estimate
            p0x_estimate = np.random.binomial(n_samples[i], eta_n*p0x + (1.0-eta_n)*0.5)/n_samples[i]
            p1x_estimate = 1.0 - p0x_estimate

            self.measurements[i] = p0_estimate
            
            # Estimate theta
            theta_estimated = np.arctan2(p0x_estimate - p1x_estimate, p0_estimate - p1_estimate)
            
            # Store this to determine angle at theta = 0 or pi/2
            if i==0:
                self.p0mp1 = p0_estimate - p1_estimate

            # Compute f(n) - Eq. 3
            fi_estimate = np.exp(1.0j*theta_estimated)
            signals[i] = fi_estimate
        
        return signals    

In [3]:
P0 = lambda n, theta: np.cos((2*n+1)*theta)**2
P1 = lambda n, theta: np.sin((2*n+1)*theta)**2

def estimate_signal(depths, n_samples, theta):
        signals = np.zeros(len(depths), dtype = np.complex128)
        measurements = np.zeros(len(depths))
        for i,n in enumerate(depths):
            # Get the exact measuremnt probabilities
            p0 = P0(n, theta)
            p1 = P1(n, theta)

            p0x = P0x(n,theta)
            p1x = P1x(n,theta)
            
            p0_estimate = np.random.binomial(n_samples[i], p0)/n_samples[i]
            p1_estimate = 1 - p0_estimate

            p0x_estimate = np.random.binomial(n_samples[i], p0x)/n_samples[i]
            p1x_estimate = 1.0 - p0x_estimate
            measurements[i] = p0_estimate
            
            # Estimate theta
            theta_estimated = np.arctan2(p0x_estimate - p1x_estimate, p0_estimate - p1_estimate)

            # Compute f(n) - Eq. 3
            fi_estimate = np.exp(1.0j*theta_estimated)
            signals[i] = fi_estimate
         
        return signals, measurements

from scipy.stats import binom

def apply_correction(ula_signal, measurements, theta_est, theta):
    theta_est = np.abs(theta_est)
    p_exact = np.cos((2*ula_signal.depths+1)*(theta))**2
    p_neg = np.cos((2*ula_signal.depths+1)*(-theta))**2
    p_o2 = np.cos((2 * ula_signal.depths + 1) * (theta_est/2.0)) ** 2
    p_o4 = np.cos((2 * ula_signal.depths + 1) * (theta_est/4.0)) ** 2
    p_same = np.cos((2*ula_signal.depths+1)*(theta_est))**2
    p_s2 = np.cos((2 * ula_signal.depths + 1) * (np.pi/2-theta_est)) ** 2
    p_s4 = np.cos((2 * ula_signal.depths + 1) * (np.pi / 4 - theta_est)) ** 2
    p_s2_o2 = np.cos((2 * ula_signal.depths + 1) * (np.pi / 2 - theta_est/2)) ** 2
    p_s4_o2 = np.cos((2 * ula_signal.depths + 1) * (np.pi / 4 - theta_est/2)) ** 2

    l_exact = np.sum(
        np.log([1e-75 + binom.pmf(ula_signal.n_samples[kk] * measurements[kk], ula_signal.n_samples[kk],
                                  p_exact[kk]) for kk in
                range(len(ula_signal.n_samples))]))
    l_neg = np.sum(
        np.log([1e-75 + binom.pmf(ula_signal.n_samples[kk] * measurements[kk], ula_signal.n_samples[kk],
                                  p_neg[kk]) for kk in
                range(len(ula_signal.n_samples))]))
    l_o2 = np.sum(
        np.log([1e-75+binom.pmf(ula_signal.n_samples[kk]*measurements[kk], ula_signal.n_samples[kk], p_o2[kk]) for kk in
         range(len(ula_signal.n_samples))]))
    l_o4 = np.sum(
        np.log([1e-75+binom.pmf(ula_signal.n_samples[kk] * measurements[kk], ula_signal.n_samples[kk], p_o4[kk]) for kk in
         range(len(ula_signal.n_samples))]))
    l_same = np.sum(
        np.log([1e-75+binom.pmf(ula_signal.n_samples[kk] * measurements[kk], ula_signal.n_samples[kk], p_same[kk]) for kk in
         range(len(ula_signal.n_samples))]))
    l_s2 = np.sum(
        np.log([1e-75+binom.pmf(ula_signal.n_samples[kk] * measurements[kk], ula_signal.n_samples[kk], p_s2[kk]) for kk in
         range(len(ula_signal.n_samples))]))
    l_s4 = np.sum(
        np.log([1e-75+binom.pmf(ula_signal.n_samples[kk] * measurements[kk], ula_signal.n_samples[kk], p_s4[kk]) for kk in
         range(len(ula_signal.n_samples))]))
    l_s2_o2 = np.sum(
        np.log([1e-75+binom.pmf(ula_signal.n_samples[kk] * measurements[kk], ula_signal.n_samples[kk], p_s2_o2[kk]) for kk in
         range(len(ula_signal.n_samples))]))
    l_s4_o2 = np.sum(
        np.log([1e-75+binom.pmf(ula_signal.n_samples[kk] * measurements[kk], ula_signal.n_samples[kk], p_s4_o2[kk]) for kk in
         range(len(ula_signal.n_samples))]))
    
    which_correction = np.argmax([l_same, l_s2, l_s4, l_o2, l_o4, l_s2_o2, l_s4_o2])
    if which_correction == 1:
        theta_est = np.pi/2.0 - theta_est
    elif which_correction == 2:
        theta_est = np.pi/4.0 - theta_est
    elif which_correction == 3:
        theta_est = theta_est/2
    elif which_correction == 4:
        theta_est = theta_est/4
    elif which_correction == 5:
        theta_est = np.pi / 2.0 - 0.5 * theta_est
    elif which_correction == 6:
        theta_est = np.pi / 4.0 - 0.5 * theta_est
    # elif which_correction == 7:
    #     theta_est = -theta_est

    # print(f'FINAL ANGLE FOUND: {theta_est, theta}')

    return np.abs(theta_est)

In [4]:
import numpy as np
import itertools
from typing import List

def generate_all_sign_variations(signs_array: np.ndarray) -> np.ndarray:
    """
    Generate all possible sign variations by changing signs at all possible pairs of positions.
    
    Parameters:
    -----------
    signs_array : numpy.ndarray
        Original array of signs (typically containing 1 or -1)
    
    Returns:
    --------
    numpy.ndarray
        Array of all possible sign variation arrays
    """
    # Get all possible unique pairs of positions
    n = len(signs_array)
    position_pairs = list(itertools.combinations(range(n), 2))
    
    # Will store all variations
    all_variations = []
    
    # Iterate through all possible position pairs
    for pos1, pos2 in position_pairs:
        # Generate variations for this pair of positions
        pair_variations = generate_pair_variations(signs_array, pos1, pos2)
        all_variations.extend(pair_variations)
    
    return all_variations

def generate_adjacent_sign_variations(signs_array: np.ndarray, size: int) -> np.ndarray:
    """
    Generate sign variations by sliding a two-position window across the array.
    
    Parameters:
    -----------
    signs_array : numpy.ndarray
        Original array of signs (typically containing 1 or -1)
    
    Returns:
    --------
    numpy.ndarray
        Array of all possible sign variation arrays
    """
    # Total length of the array
    n = len(signs_array)
    
    # Will store all variations
    all_variations = []
    
    # Iterate through adjacent position pairs 
    # (0,1), (1,2), (2,3), ... until the second-to-last pair
    for pos1 in range(1, n - size + 1):
        pos = [pos1 + i for i in range(size)]
        
        # Generate variations for this pair of adjacent positions
        pair_variations = generate_pair_variations(signs_array, pos)
 
        all_variations.extend(pair_variations)
    
    return all_variations

def generate_pair_variations(signs_array: np.ndarray, pos: List[int]) -> List[np.ndarray]:
    """
    Generate sign variations for a specific pair of positions.
    
    Parameters:
    -----------
    signs_array : numpy.ndarray
        Original array of signs
    pos1 : int
        First position to modify
    pos2 : int
        Second position to modify
    
    Returns:
    --------
    List[numpy.ndarray]
        List of sign variation arrays
    """
    # Validate input positions
    # if pos1 < 0 or pos2 < 0 or pos1 >= len(signs_array) or pos2 >= len(signs_array):
    #     raise ValueError("Positions must be within the array bounds")
    
    # Generate all possible sign combinations for the two positions
    sign_combinations = list(itertools.product([-1, 1], repeat=len(pos)))
    
    # Create variations
    variations = []
    for combo in sign_combinations:
        # Create a copy of the original array
        variation = signs_array.copy()
        
        # Modify the two specified positions
        for i in range(len(combo)):
            variation[pos[i]] *= combo[i]
        
        variations.append(tuple(variation))
    
    return variations


# Example array
signs = np.array([1]*4)

# Generate variations for all position pairs
all_variations = generate_adjacent_sign_variations(signs, 3)

# print("Original array:", signs)
# print(f"\nTotal variations: {len(all_variations)}")
# print(f'4*len(signs)^2: {4*len(signs)**2}')
# print(f'2^len(signs): {2**len(signs)}')

print("\nFirst few variations:")
for var in all_variations:  # Print first 10 variations
    print(var)


First few variations:
(1, -1, -1, -1)
(1, -1, -1, 1)
(1, -1, 1, -1)
(1, -1, 1, 1)
(1, 1, -1, -1)
(1, 1, -1, 1)
(1, 1, 1, -1)
(1, 1, 1, 1)


## Define the objective function to minimize

In [5]:
def objective_function_eig(lp, cos_signal, abs_sin, ula_signal, esprit):   
    signal = cos_signal + 1.0j * lp * abs_sin
    R = ula_signal.get_cov_matrix_toeplitz(signal)
    _, _ = esprit.estimate_theta_toeplitz(R)
    eigs = np.abs(esprit.eigs)[:2]
    obj = eigs[1] - eigs[0]

    return obj

def objective_function(lp, cos_signal, abs_sin, ula_signal, esprit):
    signal = cos_signal + 1.0j * lp * abs_sin
    R = ula_signal.get_cov_matrix_toeplitz(signal)
    theta_est, _ = esprit.estimate_theta_toeplitz(R)

    theta_est = apply_correction(ula_signal, ula_signal.measurements, theta_est, theta_est)

    # print(f'2*theta_est: {2*theta_est}')
    p_same = np.cos((2 * ula_signal.depths + 1) * (theta_est)) ** 2

    obj = -np.sum(
        np.log(
            [1e-75 + binom.pmf(ula_signal.n_samples[kk] * ula_signal.measurements[kk], ula_signal.n_samples[kk], p_same[kk]) for kk
             in
             range(len(ula_signal.n_samples))]))


    # eigs = np.abs(esprit.eigs)[:2]
    # obj = eigs[1] - eigs[0]

    return obj

## What is the distribution of the signs? 

In [6]:
narray = [2,2,2,2,2,3]

def get_heavy_signs(narray, steps):

    thetas = np.linspace(np.arcsin(0.09), np.arcsin(0.91), steps)
    avals = np.sin(thetas)

    ula_signal = TwoqULASignal(M=narray, C=3)

    num_mc = 1000

    sign_distributions = {}

    for a,theta in zip(avals,thetas):
        # theta = np.arcsin(a)
        distr = {}
        for i in range(num_mc):
            signal, _ = estimate_signal(ula_signal.depths, ula_signal.n_samples, theta)
                # print(measurements)
            signs = tuple(np.sign(np.imag(signal)))
            # print(signs)

            distr[signs] = distr.get(signs, 0.0) + 1/num_mc
        sign_distributions[str(a)] = distr

    def get_signs(sign_distribution):
        # Sort the dictionary by values in descending order
        sorted_data = dict(sorted(sign_distribution.items(), key=lambda item: item[1], reverse=True))

        # Create a new dictionary with cumulative sum close to 0.68
        cumulative_dict = {}
        cumulative_sum = 0.0

        for key, value in sorted_data.items():
            cumulative_dict[key] = value
            if cumulative_sum + value > 0.9:
                break
            cumulative_sum += value

        # Output the result
        return cumulative_dict
    
    ret_dict = {}
    for a in avals:
        distr = get_signs(sign_distributions[str(a)])
        ret_dict[str(a)] = distr
 
    # Normalize the values
    normalized_data = {}
    for key, sub_dict in ret_dict.items():
        total_sum = sum(sub_dict.values())
        normalized_data[key] = {k: v / total_sum for k, v in sub_dict.items()}

    return normalized_data

normalized_data = get_heavy_signs(narray, 10)

# normalized_data

In [7]:
import ast
# Taking a random sample based on the normalized probabilities
def sample_from_normalized(data, sample_size=1):
    keys = list(map(str, list(data.keys())))
    probabilities = list(data.values())
    sampled_keys = np.random.choice(a=keys, size=sample_size, p=probabilities)
    return [ast.literal_eval(t) for t in sampled_keys]
avals = list(normalized_data.keys())
# Get one random sample
sampled = sample_from_normalized(normalized_data[avals[0]], sample_size=3)
# print("Sampled keys:",sampled)

## Use the distribution of signs so we don't have to try all possible signs

In [8]:
np.random.seed(42)

a = 0.1
print(a)
theta = np.arcsin(a)

narray = [2,2,2,2,2,2,2,2]
heavy_signs = get_heavy_signs(narray, len(narray)**2)

ula_signal = TwoqULASignal(M=narray, C=5)
esprit = ESPIRIT()

def sample_signs(heavy_signs, sample_size=3):
    """
    Sample a set of sign variations from a learned sign distribution.

    Args:
        heavy_signs (dict): A dictionary where the keys are strings representing float values,
                           and the values are dictionaries with keys as sign vectors and values
                           as their corresponding probabilities.
        sample_size (int, optional): The number of sign variations to sample. Defaults to 3.

    Returns:
        dict: A dictionary where the keys are the same as the keys in `heavy_signs`,
              and the values are lists of sampled sign variations.
    """
    avals = list(heavy_signs.keys())
    signs_to_try = {}
    for a in avals:
        signs_to_try[a] = list(set(sample_from_normalized(heavy_signs[a], sample_size=sample_size)))

    return signs_to_try

def avals_to_usef(a0, avals, L=3):
    """
    Find the `L` values in `avals` that are closest to the given `a0` value.

    Args:
        a0 (float): The reference value to find the closest `L` values for.
        avals (list): A list of float values representing the keys in `heavy_signs`.
        L (int, optional): The number of closest values to return. Defaults to 3.

    Returns:
        list: A list of string representations of the `L` closest values to `a0` in `avals`.
    """
    avals = list(map(float, avals))

    left = 0
    right = len(avals)
    
    while left < right:
        mid = (left + right) // 2
        
        if avals[mid] <= a0:
            left = mid + 1
        else:
            right = mid

    if left >= 4:
        avals_to_use = list(map(str, avals[left-L: left+L]))
    if left >= 3:
        avals_to_use = list(map(str, avals[left-3: left+L]))
    elif left>=2:
        avals_to_use = list(map(str, avals[left-2: left+L]))
    elif left>=1:
        avals_to_use = list(map(str, avals[left-1: left+L]))
    else:
        avals_to_use = list(map(str, avals[left: left+L]))
    
    return avals_to_use

def all_signs_to_try(avals_to_use, signs_to_try, adjacency=2):
    """
    Generate all possible sign variations within a Hamming distance of 2 from the given `avals_to_use`.

    Args:
        avals_to_use (list): A list of string representations of float values.
        signs_to_try (dict): A dictionary where the keys are string representations of float values,
                            and the values are lists of sign variations.
        adjacency (int, optional): The maximum Hamming distance to consider for the sign variations.
                                  Defaults to 2.

    Returns:
        list: A list of all possible sign variations within the specified Hamming distance.
    """
    all_signs = []
    for a in avals_to_use:
        for x in signs_to_try[a]:
            hamming_distance_two_signs = generate_adjacent_sign_variations(np.array(x), adjacency)
            all_signs.extend(hamming_distance_two_signs)
    all_signs = list(set(all_signs))
    
    return all_signs

def minimize_obj(all_signs, cos_signal, abs_sin, ula_signal, esprit, disp):
    """
    Find the sign variation that minimizes the objective function.

    Args:
        all_signs (list): A list of all possible sign variations to consider.
        cos_signal (numpy.ndarray): The real part of the estimated signal.
        abs_sin (numpy.ndarray): The absolute value of the imaginary part of the estimated signal.
        ula_signal (ULASignal): An instance of the ULASignal class, containing information about the signal.
        esprit (ESPRIT): An instance of the ESPRIT class, used for estimating the signal parameters.
        disp (bool): If True, prints the current objective and the current best sign variation.

    Returns:
        numpy.ndarray: The sign variation that minimizes the objective function.
    """
    # obj = 1.0
    x_star = np.array(all_signs[0])
    obj = objective_function(x_star, cos_signal, abs_sin, ula_signal, esprit)
    for x in all_signs:
        curr_obj = objective_function(np.array(x), cos_signal, abs_sin, ula_signal, esprit)
        if curr_obj < obj:
            if disp:
                print(f'current objective: {curr_obj}')
                print(f'current best signs: {x}')
            obj = curr_obj
            x_star = np.array(x)

    return x_star

def csae_with_local_minimization(theta, ula_signal, esprit, heavy_signs, sample=False, correction=False, optimize=False, disp=False):
    """
    Perform CSAE (Compressive Sensing Angle Estimation) with local minimization.

    Args:
        theta (float): The true angle of the signal.
        ula_signal (ULASignal): An instance of the ULASignal class, containing information about the signal.
        esprit (ESPRIT): An instance of the ESPRIT class, used for estimating the signal parameters.
        heavy_signs (dict): A dictionary where the keys are strings representing float values,
                           and the values are dictionaries with keys as sign vectors and values
                           as their corresponding probabilities.
        sample (bool, optional): If True, samples sign variations from the learned sign distribution.
                                If False, uses the learned sign distribution directly. Defaults to False.
        correction (bool, optional): If True, applies a correction to the estimated angle. Defaults to False.
        optimize (bool, optional): If True, uses a sliding window approach to find the best sign variation.
                                  If False, uses the learned sign distribution directly. Defaults to False.
        disp (bool, optional): If True, prints additional information during the process. Defaults to False.

    Returns:
        dict: A dictionary containing the estimated angles, errors, number of queries, maximum depth,
              and other relevant information.
    """

    depths = ula_signal.depths
    n_samples = ula_signal.n_samples
    csignal, measurements = estimate_signal(depths, n_samples, theta)
    ula_signal.measurements = measurements

    cos_signal = np.real(csignal)

    correct_signs = np.sign(np.imag(csignal))
    if disp:
        print(f'correct signs: {correct_signs}')
    abs_sin = np.abs(np.imag(csignal))

    if sample:
        # step 1: sample signs from learned sign distribution
        signs_to_try = sample_signs(heavy_signs=heavy_signs, sample_size=3)
        avals = list(heavy_signs.keys())

        if optimize:
            # step 2: using rough estimate where the amplitude is, use the a-values around that estimate
            a0 = np.sqrt(0.5 - 0.5 * cos_signal[0])
            avals_to_use = avals_to_usef(a0, avals, L=3)

            if disp:
                print(f'rough estimate a: {a0}')
                print(f'avals to use: {avals_to_use}')

            # step 3: now vary the signs in a sliding window of size "adjacency"
            all_signs = all_signs_to_try(avals_to_use, signs_to_try, adjacency=2)

            if disp:
                print(f'number of signs Hamming distance two: {len(all_signs)}')

            # step 4: try all the signs pick the ones that minimize the objective function
            if disp:
                print('debug')
                print(all_signs)
            x_star = minimize_obj(all_signs, cos_signal, abs_sin, ula_signal, esprit, disp)

            # step 5 (optional): do one more sweep
            hamming_distance_one_signs = list(set(generate_adjacent_sign_variations(x_star, 1)))
            x_star = minimize_obj(hamming_distance_one_signs, cos_signal, abs_sin, ula_signal, esprit, disp)
        
        else:
            # here we don't vary the signs using a sliding window, but directly use the learned sign distribution (poor performance)
            all_signs = []
            for a in avals:
                all_signs.extend(heavy_signs[a])
            all_signs = list(set(all_signs))
            x_star = minimize_obj(all_signs, cos_signal, abs_sin, ula_signal, esprit, disp)

    else:
        avals = list(heavy_signs.keys())
        signs_to_try = {}
        for a in avals:
            signs_to_try[a] = heavy_signs[a]

        if optimize:
            # step 2: using rough estimate where the amplitude is, use the a-values around that estimate
            a0 = np.sqrt(0.5 - 0.5 * cos_signal[0])
            avals_to_use = avals_to_usef(a0, avals, L=3)

            if disp:
                print(f'rough estimate a: {a0}')
                print(f'avals to use: {avals_to_use}')

            # step 3: now vary the signs in a sliding window of size "adjacency"
            all_signs = all_signs_to_try(avals_to_use, signs_to_try, adjacency=2)

            if disp:
                print(f'number of signs Hamming distance two: {len(all_signs)}')

            # step 4: try all the signs pick the ones that minimize the objective function
            x_star = minimize_obj(all_signs, cos_signal, abs_sin, ula_signal, esprit, disp, measu)

            # step 5 (optional): do one more sweep
            hamming_distance_one_signs = list(set(generate_adjacent_sign_variations(x_star, 1)))
            x_star = minimize_obj(hamming_distance_one_signs, cos_signal, abs_sin, ula_signal, esprit, disp)
        
        else:
            # here we don't vary the signs using a sliding window, but directly use the learned sign distribution (poor performance)
            all_signs = []
            for a in avals:
                all_signs.extend(heavy_signs[a])
            all_signs = list(set(all_signs))
            x_star = minimize_obj(all_signs, cos_signal, abs_sin, ula_signal, esprit, disp)


    # Optimization is done.

    if disp:
        print(x_star)
        
    signal = cos_signal + 1.0j * x_star * abs_sin
    R = ula_signal.get_cov_matrix_toeplitz(signal)
    theta_est, _ = esprit.estimate_theta_toeplitz(R)
    theta_est = np.abs(theta_est)
    eigs = np.abs(esprit.eigs)[:2]
    basin_obj = eigs[1] - eigs[0]
    if correction:
        theta_est = apply_correction(ula_signal, measurements, theta_est, theta) #apply correction
    if disp:
        print(f'basin obj: {basin_obj}')

    cR = ula_signal.get_cov_matrix_toeplitz(csignal)
    theta_est1, _ = esprit.estimate_theta_toeplitz(cR)
    eigs = np.abs(esprit.eigs)[:2]
    true_obj = eigs[1] - eigs[0]
    if disp:
        print(f'true obj: {true_obj}')

    # compute queries required
    num_queries = np.sum(np.array(ula_signal.depths)*np.array(ula_signal.n_samples)) + ula_signal.n_samples[0]
    max_single_query = np.max(ula_signal.depths)

    ret_dict = {'theta_est': theta_est, 'theta_est1': theta_est1, 
                'error': np.abs(np.sin(theta)-np.sin(theta_est)), 
                'error1': np.abs(np.sin(theta)-np.sin(theta_est1)), 
                'queries': num_queries, 'depth': max_single_query,
                'true_obj': true_obj, 'basin_obj': basin_obj,
                'x_star': x_star}
    
    return ret_dict

csae_with_local_minimization(theta, ula_signal, esprit, heavy_signs, sample=True, correction=True, optimize=True, disp=True)

0.1
correct signs: [ 1.  1.  1.  1. -1.  1.  1.  1.  1.]
rough estimate a: 0.18910752115495127
avals to use: ['0.13981296510949756', '0.15634532387578837', '0.1728339924311597', '0.18927436306884374', '0.20566184157877418', '0.22199184853142423']
number of signs Hamming distance two: 114
debug
[(1.0, -1.0, -1.0, 1.0, -1.0, 1.0, -1.0, -1.0, 1.0), (1.0, 1.0, 1.0, -1.0, -1.0, -1.0, -1.0, 1.0, 1.0), (1.0, 1.0, 1.0, -1.0, 1.0, 1.0, -1.0, -1.0, -1.0), (1.0, 1.0, 1.0, 1.0, 1.0, -1.0, -1.0, 1.0, 1.0), (1.0, 1.0, 1.0, 1.0, -1.0, 1.0, 1.0, -1.0, -1.0), (1.0, 1.0, 1.0, 1.0, -1.0, -1.0, 1.0, -1.0, -1.0), (1.0, 1.0, 1.0, 1.0, 1.0, -1.0, 1.0, 1.0, 1.0), (1.0, 1.0, -1.0, -1.0, 1.0, 1.0, -1.0, 1.0, 1.0), (1.0, 1.0, -1.0, -1.0, 1.0, -1.0, -1.0, 1.0, 1.0), (1.0, 1.0, 1.0, -1.0, 1.0, -1.0, -1.0, 1.0, 1.0), (1.0, 1.0, 1.0, -1.0, -1.0, 1.0, 1.0, 1.0, -1.0), (1.0, 1.0, -1.0, -1.0, -1.0, -1.0, 1.0, 1.0, -1.0), (1.0, 1.0, 1.0, 1.0, -1.0, 0.0, 1.0, -1.0, -1.0), (1.0, 1.0, 1.0, 1.0, -1.0, 1.0, -1.0, -1.0, 1.0),

{'theta_est': 0.10014050463580293,
 'theta_est1': 0.10012026433053867,
 'error': 2.6781641200374073e-05,
 'error1': 4.692056561073077e-05,
 'queries': 2600,
 'depth': 128,
 'true_obj': -397.45165984862285,
 'basin_obj': -71.49676886330401,
 'x_star': array([ 1., -1.,  1.,  1.,  1., -1.,  1., -1.,  1.])}

## Let's do some statistics now

In [9]:
a0=0.14658716786698373
signs_to_try = sample_signs(heavy_signs=heavy_signs, sample_size=3)
avals = list(heavy_signs.keys())
avals_to_usef(a0, avals, L=3), avals, a0

(['0.10663566752567158',
  '0.12324153604814797',
  '0.13981296510949756',
  '0.15634532387578837',
  '0.1728339924311597',
  '0.18927436306884374'],
 ['0.09',
  '0.10663566752567158',
  '0.12324153604814797',
  '0.13981296510949756',
  '0.15634532387578837',
  '0.1728339924311597',
  '0.18927436306884374',
  '0.20566184157877418',
  '0.22199184853142423',
  '0.23825982055751374',
  '0.2544612116232283',
  '0.2705914943005945',
  '0.286646161032655',
  '0.3026207253930916',
  '0.318510723339943',
  '0.33431171446306707',
  '0.35001928322499964',
  '0.3656290401948629',
  '0.3811366232749777',
  '0.3965376989198384',
  '0.41182796334710836',
  '0.42700314374029846',
  '0.44205899944279253',
  '0.4569913231428852',
  '0.47179594204950215',
  '0.4864687190582737',
  '0.5010055539076351',
  '0.5154023843246325',
  '0.5296551871601116',
  '0.5437599795129751',
  '0.5577128198431908',
  '0.5715098090732432',
  '0.5851470916777191',
  '0.5986208567607206',
  '0.6119273391208092',
  '0.6250628

In [10]:
np.random.seed(42)

avals = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
# avals = [np.random.uniform(0.1, 0.9)]
# avals = [0.5, 0.6, 0.7, 0.8, 0.9]
# avals = [0.1, 0.2, 0.3]
avals = [np.random.uniform(0.1, 0.9) for _ in range(15)]
# narray = [2]*5 + [3]
narray = [2]*4

print('learning the distribution of signs...')
heavy_signs = get_heavy_signs(narray, int(6*len(narray)))
num_queries = np.zeros(len(avals), dtype=int)
max_single_query = np.zeros(len(avals), dtype=int)

num_mc=100
thetas = np.zeros((len(avals), num_mc))
errors = np.zeros((len(avals), num_mc))
thetas1 = np.zeros((len(avals), num_mc))
errors1 = np.zeros((len(avals), num_mc))

basin_obj = np.zeros((len(avals), num_mc))
true_obj = np.zeros((len(avals), num_mc))

ula_signal = TwoqULASignal(M=narray, C=5)
esprit = ESPIRIT()

for j,a in enumerate(avals):
    theta = np.arcsin(a)
    print(f'theta = {theta}')
    disp=False
    # if j==4:
    #     disp=True
    for i in tqdm(range(num_mc)):
        res = csae_with_local_minimization(theta, ula_signal, esprit, heavy_signs, sample=True, correction=True, optimize=True, disp=disp)
        thetas[j][i] = res['theta_est']
        errors[j][i] = res['error']

        thetas1[j][i] = res['theta_est1']
        errors1[j][i] = res['error1']

        basin_obj[j][i] = res['basin_obj']
        true_obj[j][i] = res['true_obj']
    num_queries[j] = res['queries']
    max_single_query[j] = res['depth']

    print(f'constant factors query and depth: {np.percentile(errors[j], 95):.3e}, {np.percentile(errors[j], 95) * num_queries[j]}, {np.percentile(errors[j], 95) * max_single_query[j]}')

learning the distribution of signs...
theta = 0.41111546403370536


  0%|          | 0/100 [00:00<?, ?it/s]

constant factors query and depth: 1.870e-02, 3.3666359341273115, 0.14962826373899163
theta = 1.0363905664439828


  0%|          | 0/100 [00:00<?, ?it/s]

constant factors query and depth: 1.845e-02, 3.3204706104907347, 0.14757647157736598
theta = 0.7554209219922304


...

constant factors query and depth: 1.569e-02, 2.8234565645122505, 0.1254869584227667
theta = 0.2732593578259982


  0%|          | 0/100 [00:00<?, ?it/s]

constant factors query and depth: 1.489e-02, 2.6806425203444593, 0.11913966757086486
theta = 0.24799415401854524


  0%|          | 0/100 [00:00<?, ?it/s]

constant factors query and depth: 2.631e-02, 4.736160113653244, 0.2104960050512553


In [11]:
for i in range(len(avals)):
    print(np.sum(np.where(np.array(basin_obj[i])/np.array(true_obj[i]) >= 0.99, 1, 0))/len(basin_obj[i]))

0.14
0.2
0.25
0.17
0.14
0.16
0.2
0.15
0.26
0.06
0.32
0.53
0.51
0.19
0.27


In [12]:
i=6
np.array(basin_obj[i])/np.array(true_obj[i]), np.sum(np.where(np.array(basin_obj[i])/np.array(true_obj[i]) >= 0.99, 1, 0))/len(basin_obj[i])

(array([0.16005468, 0.88041744, 0.35395624, 0.33454116, 0.30185703,
        0.96348892, 0.23848   , 1.        , 0.09394278, 1.        ,
        0.65911071, 0.22377447, 0.16903925, 0.23219455, 0.10760314,
        0.12192419, 0.15734696, 1.        , 0.13953991, 0.07784626,
        0.40005274, 0.57562107, 0.36711104, 0.10934473, 0.12482727,
        1.        , 0.16066305, 1.        , 0.12447119, 1.        ,
        0.08967808, 0.27506446, 0.24136119, 0.356828  , 0.73807222,
        0.10263149, 1.        , 0.21624037, 0.59955709, 1.        ,
        1.        , 0.18585462, 0.09777969, 1.05938911, 0.09269206,
        0.25974478, 0.41457943, 0.09774346, 0.6933519 , 1.        ,
        1.        , 0.23604582, 0.18619113, 0.57794034, 0.07821842,
        0.08173108, 0.78331883, 0.19093583, 0.15863997, 1.        ,
        0.89221044, 0.15049113, 0.13158803, 1.        , 0.11066742,
        0.9110941 , 0.18541151, 0.62664092, 0.44554577, 0.2018394 ,
        0.49087274, 0.09769741, 1.        , 0.38

In [13]:
i=6
np.sin(thetas[i]), avals[i]

(array([0.14775781, 0.1385406 , 0.13279854, 0.13854673, 0.15059242,
        0.13210707, 0.1449994 , 0.14175501, 0.1923323 , 0.14008262,
        0.15574162, 0.15507179, 0.13189001, 0.14163192, 0.13663395,
        0.13502671, 0.13916812, 0.16010863, 0.13935168, 0.14871157,
        0.13694166, 0.15062664, 0.1327524 , 0.1476518 , 0.14788026,
        0.14261911, 0.14514006, 0.14270738, 0.15217976, 0.13609439,
        0.15013154, 0.15288383, 0.14863345, 0.13855705, 0.17413922,
        0.16497217, 0.14532673, 0.13768693, 0.15383773, 0.13431567,
        0.15772506, 0.13724876, 0.15408273, 0.13543123, 0.17812518,
        0.15844732, 0.15798714, 0.13998335, 0.15912292, 0.14647185,
        0.14626734, 0.17314526, 0.15946675, 0.17129441, 0.16978222,
        0.14619162, 0.16577349, 0.15085871, 0.14823757, 0.13763132,
        0.13055231, 0.15194661, 0.15906938, 0.14488303, 0.16161299,
        0.13987151, 0.14435883, 0.16617878, 0.13399038, 0.12504971,
        0.14961745, 0.11692923, 0.15720603, 0.13

In [14]:
i=6
thetas[i]/np.pi, np.arcsin(avals[i])/np.pi

(array([0.04720561, 0.04424114, 0.04239633, 0.04424311, 0.04811811,
        0.04217427, 0.04631803, 0.04527452, 0.06160514, 0.04473682,
        0.04977672, 0.04956089, 0.04210457, 0.04523494, 0.04362841,
        0.04311202, 0.04444284, 0.05118445, 0.04450185, 0.0475126 ,
        0.04372729, 0.04812913, 0.04238151, 0.0471715 , 0.04724503,
        0.0455524 , 0.04636328, 0.04558078, 0.04862927, 0.04345504,
        0.04796972, 0.04885603, 0.04748745, 0.04424643, 0.05571428,
        0.05275343, 0.04642333, 0.04396678, 0.0491633 , 0.04288361,
        0.05041597, 0.04382597, 0.04924223, 0.04324198, 0.0570032 ,
        0.0506488 , 0.05050045, 0.04470491, 0.05086662, 0.04679178,
        0.04672597, 0.05539301, 0.05097748, 0.05479493, 0.05430643,
        0.04670161, 0.05301206, 0.04820385, 0.04736003, 0.04394891,
        0.04167505, 0.04855418, 0.05084936, 0.04628059, 0.05166963,
        0.04466895, 0.04611196, 0.05314288, 0.04277912, 0.03990903,
        0.04780421, 0.03730507, 0.05024867, 0.04

In [15]:
# np.random.seed(1)

# csignal, measurements = estimate_signal(ula_signal.depths, ula_signal.n_samples, np.arcsin(avals[i]))

# plt.plot(measurements, 'r', label='measurements')
# plt.plot(np.cos((2*ula_signal.depths + 1)*theta2)**2, 'b', label='theta2')
# plt.plot(np.cos((2*ula_signal.depths + 1)*theta1)**2, 'k', label='theta1')
# plt.legend()

In [16]:
perc = 68
avg = 0.0
avg_err = 0.0
for i in range(len(avals)):
    avg += np.percentile(errors[i], perc) * num_queries[i]/len(avals)
    avg_err += np.percentile(errors[i], perc)/len(avals)
    print(f'constant factors query and depth ({perc}%) for {avals[i]:.3f}: {np.percentile(errors[i], perc):.3e}, {np.percentile(errors[i], perc) * num_queries[i]:.3f}, {np.percentile(errors[i], perc) * max_single_query[i]:.3e}')
print(f'average constant factor {avg:0.3f}')
print(f'average error {avg_err:0.3e}')

constant factors query and depth (68%) for 0.400: 8.843e-03, 1.592, 7.075e-02
constant factors query and depth (68%) for 0.861: 5.419e-03, 0.975, 4.335e-02
constant factors query and depth (68%) for 0.686: 7.632e-03, 1.374, 6.105e-02
constant factors query and depth (68%) for 0.579: 1.010e-02, 1.818, 8.079e-02
constant factors query and depth (68%) for 0.225: 1.043e-02, 1.878, 8.347e-02
constant factors query and depth (68%) for 0.225: 9.398e-03, 1.692, 7.518e-02
constant factors query and depth (68%) for 0.146: 1.204e-02, 2.166, 9.628e-02
constant factors query and depth (68%) for 0.793: 6.483e-03, 1.167, 5.186e-02
constant factors query and depth (68%) for 0.581: 1.077e-02, 1.939, 8.617e-02
constant factors query and depth (68%) for 0.666: 7.219e-03, 1.300, 5.776e-02
constant factors query and depth (68%) for 0.116: 9.712e-03, 1.748, 7.769e-02
constant factors query and depth (68%) for 0.876: 6.561e-03, 1.181, 5.249e-02
constant factors query and depth (68%) for 0.766: 8.636e-03, 1.5

In [17]:
perc = 68
avg = 0.0
avg_err = 0.0
for j in range(len(avals)):
    avg += 2*np.percentile(errors1[j], perc) * num_queries[j]/len(avals)
    avg_err += np.percentile(errors1[j], perc)/len(avals)
    print(f'constant factors query and depth with known signs for {avals[j]:.3f}: {2*np.percentile(errors1[j], perc):.3e}, {2*np.percentile(errors1[j], perc) * num_queries[j]:.3f}, {2*np.percentile(errors1[j], perc) * max_single_query[j]:.3e}')
print(f'average constant factor {avg:0.3f}')
print(f'average error {avg_err:0.3e}')

constant factors query and depth with known signs for 0.400: 2.481e-02, 4.466, 1.985e-01
constant factors query and depth with known signs for 0.861: 1.184e-02, 2.131, 9.472e-02
constant factors query and depth with known signs for 0.686: 1.836e-02, 3.305, 1.469e-01
constant factors query and depth with known signs for 0.579: 1.739e-02, 3.131, 1.392e-01
constant factors query and depth with known signs for 0.225: 2.835e-02, 5.103, 2.268e-01
constant factors query and depth with known signs for 0.225: 2.449e-02, 4.408, 1.959e-01
constant factors query and depth with known signs for 0.146: 3.186e-02, 5.735, 2.549e-01
constant factors query and depth with known signs for 0.793: 1.582e-02, 2.847, 1.265e-01
constant factors query and depth with known signs for 0.581: 2.258e-02, 4.065, 1.806e-01
constant factors query and depth with known signs for 0.666: 1.712e-02, 3.082, 1.370e-01
constant factors query and depth with known signs for 0.116: 2.238e-02, 4.028, 1.790e-01
constant factors quer

In [18]:
perc = 95
avg = 0.0
avg_err = 0.0
for i in range(len(avals)):
    avg += np.percentile(errors[i], perc) * num_queries[i]/len(avals)
    avg_err += np.percentile(errors[i], perc)/len(avals)
    print(f'constant factors query and depth ({perc}%) for {avals[i]:.3f}: {np.percentile(errors[i], perc):.3e}, {np.percentile(errors[i], perc) * num_queries[i]:.3f}, {np.percentile(errors[i], perc) * max_single_query[i]:.3e}')
print(f'average constant factor {avg:0.3f}')
print(f'average error {avg_err:0.3e}')

constant factors query and depth (95%) for 0.400: 1.870e-02, 3.367, 1.496e-01
constant factors query and depth (95%) for 0.861: 1.845e-02, 3.320, 1.476e-01
constant factors query and depth (95%) for 0.686: 2.636e-02, 4.745, 2.109e-01
constant factors query and depth (95%) for 0.579: 2.105e-02, 3.789, 1.684e-01
constant factors query and depth (95%) for 0.225: 2.117e-02, 3.811, 1.694e-01
constant factors query and depth (95%) for 0.225: 2.040e-02, 3.673, 1.632e-01
constant factors query and depth (95%) for 0.146: 2.777e-02, 4.998, 2.221e-01
constant factors query and depth (95%) for 0.793: 1.403e-02, 2.526, 1.122e-01
constant factors query and depth (95%) for 0.581: 2.288e-02, 4.118, 1.830e-01
constant factors query and depth (95%) for 0.666: 1.749e-02, 3.148, 1.399e-01
constant factors query and depth (95%) for 0.116: 3.377e-02, 6.078, 2.701e-01
constant factors query and depth (95%) for 0.876: 1.363e-02, 2.454, 1.091e-01
constant factors query and depth (95%) for 0.766: 1.569e-02, 2.8

In [19]:
perc = 95
avg = 0.0
avg_err = 0.0
for j in range(len(avals)):
    avg += 2*np.percentile(errors1[j], perc) * num_queries[j]/len(avals)
    avg_err += np.percentile(errors1[j], perc)/len(avals)
    print(f'constant factors query and depth with known signs for {avals[j]:.3f}: {2*np.percentile(errors1[j], perc):.3e}, {2*np.percentile(errors1[j], perc) * num_queries[j]:.3f}, {2*np.percentile(errors1[j], perc) * max_single_query[j]:.3e}')
print(f'average constant factor {avg:0.3f}')
print(f'average error {avg_err:0.3e}')

constant factors query and depth with known signs for 0.400: 4.886e-02, 8.796, 3.909e-01
constant factors query and depth with known signs for 0.861: 2.967e-02, 5.340, 2.373e-01
constant factors query and depth with known signs for 0.686: 3.615e-02, 6.508, 2.892e-01
constant factors query and depth with known signs for 0.579: 2.941e-02, 5.293, 2.353e-01
constant factors query and depth with known signs for 0.225: 5.021e-02, 9.039, 4.017e-01
constant factors query and depth with known signs for 0.225: 4.542e-02, 8.175, 3.633e-01
constant factors query and depth with known signs for 0.146: 5.478e-02, 9.860, 4.382e-01
constant factors query and depth with known signs for 0.793: 2.711e-02, 4.879, 2.168e-01
constant factors query and depth with known signs for 0.581: 4.775e-02, 8.596, 3.820e-01
constant factors query and depth with known signs for 0.666: 3.631e-02, 6.536, 2.905e-01
constant factors query and depth with known signs for 0.116: 4.484e-02, 8.071, 3.587e-01
constant factors quer

In [20]:
perc = 99
avg = 0.0
avg_err = 0.0
for i in range(len(avals)):
    avg += np.percentile(errors[i], perc) * num_queries[i]/len(avals)
    avg_err += np.percentile(errors[i], perc)/len(avals)
    print(f'constant factors query and depth ({perc}%) for {avals[i]:.3f}: {np.percentile(errors[i], perc):.3e}, {np.percentile(errors[i], perc) * num_queries[i]:.3f}, {np.percentile(errors[i], perc) * max_single_query[i]:.3e}')
print(f'average constant factor {avg:0.3f}')
print(f'average error {avg_err:0.3e}')

constant factors query and depth (99%) for 0.400: 2.347e-02, 4.225, 1.878e-01
constant factors query and depth (99%) for 0.861: 2.481e-02, 4.466, 1.985e-01
constant factors query and depth (99%) for 0.686: 2.837e-01, 51.066, 2.270e+00
constant factors query and depth (99%) for 0.579: 4.414e-02, 7.945, 3.531e-01
constant factors query and depth (99%) for 0.225: 3.533e-02, 6.359, 2.826e-01
constant factors query and depth (99%) for 0.225: 2.441e-02, 4.394, 1.953e-01
constant factors query and depth (99%) for 0.146: 4.617e-02, 8.311, 3.694e-01
constant factors query and depth (99%) for 0.793: 1.916e-02, 3.449, 1.533e-01
constant factors query and depth (99%) for 0.581: 3.769e-02, 6.784, 3.015e-01
constant factors query and depth (99%) for 0.666: 2.066e-02, 3.719, 1.653e-01
constant factors query and depth (99%) for 0.116: 3.784e-02, 6.811, 3.027e-01
constant factors query and depth (99%) for 0.876: 1.636e-02, 2.944, 1.308e-01
constant factors query and depth (99%) for 0.766: 1.760e-02, 3.

In [21]:
perc = 99
avg = 0.0
avg_err = 0.0
for j in range(len(avals)):
    avg += 2*np.percentile(errors1[j], perc) * num_queries[j]/len(avals)
    avg_err += np.percentile(errors1[j], perc)/len(avals)
    print(f'constant factors query and depth with known signs for {avals[j]:.3f}: {2*np.percentile(errors1[j], perc):.3e}, {2*np.percentile(errors1[j], perc) * num_queries[j]:.3f}, {2*np.percentile(errors1[j], perc) * max_single_query[j]:.3e}')
print(f'average constant factor {avg:0.3f}')
print(f'average error {avg_err:0.3e}')

constant factors query and depth with known signs for 0.400: 5.561e-02, 10.009, 4.449e-01
constant factors query and depth with known signs for 0.861: 5.137e-02, 9.246, 4.109e-01
constant factors query and depth with known signs for 0.686: 4.629e-02, 8.332, 3.703e-01
constant factors query and depth with known signs for 0.579: 6.906e-02, 12.431, 5.525e-01
constant factors query and depth with known signs for 0.225: 6.546e-02, 11.783, 5.237e-01
constant factors query and depth with known signs for 0.225: 5.062e-02, 9.111, 4.049e-01
constant factors query and depth with known signs for 0.146: 6.784e-02, 12.211, 5.427e-01
constant factors query and depth with known signs for 0.793: 4.553e-02, 8.195, 3.642e-01
constant factors query and depth with known signs for 0.581: 6.841e-02, 12.313, 5.473e-01
constant factors query and depth with known signs for 0.666: 4.665e-02, 8.397, 3.732e-01
constant factors query and depth with known signs for 0.116: 7.003e-02, 12.606, 5.603e-01
constant factor